## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"

df = spark.read.csv(file_location, header= True, inferSchema = True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



total bill is dependent feature/ target variable

In [0]:
df.columns

Out[5]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

## Categorical features

#### Encoding

In [0]:
# ordinal encoding
from pyspark.ml.feature import StringIndexer

In [0]:
# for a single column
indexer = StringIndexer(inputCol = 'sex', outputCol= 'sex_indexed')
df_indexed = indexer.fit(df).transform(df)
df_indexed.show()


+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
# for multiple column
indexer = StringIndexer(inputCols=['smoker','day','time'], outputCols = ['smoker_indexed','day_indexed','time_indexed'])
df_indexed = indexer.fit(df_indexed).transform(df_indexed)
df_indexed.show()


+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


## Group Independent features

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
input_cols = ['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed']

In [0]:
feature_assembler = VectorAssembler(inputCols = input_cols, outputCol = 'independent features')

In [0]:
feature_assembler_output = feature_assembler.transform(df_indexed)
feature_assembler_output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|independent features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
feature_assembler_output.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- sex_indexed: double (nullable = false)
 |-- smoker_indexed: double (nullable = false)
 |-- day_indexed: double (nullable = false)
 |-- time_indexed: double (nullable = false)
 |-- independent features: vector (nullable = true)



In [0]:
feature_assembler_output.select('independent features').show()

+--------------------+
|independent features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
grouped_df = feature_assembler_output.select('independent features','total_bill')
grouped_df.show()

+--------------------+----------+
|independent features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



## Dataset Split

In [0]:
train_df,test_df = grouped_df.randomSplit([.75,.25])

In [0]:
train_df.show()

+--------------------+----------+
|independent features|total_bill|
+--------------------+----------+
|(6,[0,1],[1.25,2.0])|     10.51|
|(6,[0,1],[1.45,2.0])|      9.55|
|(6,[0,1],[1.75,2.0])|     17.82|
| (6,[0,1],[2.0,2.0])|     13.37|
|(6,[0,1],[2.01,2.0])|     20.23|
|(6,[0,1],[2.24,3.0])|     16.04|
|(6,[0,1],[2.31,3.0])|     18.69|
| (6,[0,1],[2.5,4.0])|     18.35|
|(6,[0,1],[2.64,3.0])|     17.59|
| (6,[0,1],[3.0,4.0])|     20.45|
|(6,[0,1],[3.15,3.0])|     20.08|
|(6,[0,1],[3.18,2.0])|     19.82|
|(6,[0,1],[3.27,2.0])|     17.78|
|(6,[0,1],[3.35,3.0])|     20.65|
|(6,[0,1],[3.39,2.0])|     11.61|
| (6,[0,1],[3.6,3.0])|     24.06|
|(6,[0,1],[3.76,2.0])|     18.24|
|(6,[0,1],[4.08,2.0])|     17.92|
| (6,[0,1],[4.3,2.0])|      21.7|
| (6,[0,1],[5.0,3.0])|     31.27|
+--------------------+----------+
only showing top 20 rows



## Regression Model

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
# instantiate
linear_regressor = LinearRegression(featuresCol= 'independent features', labelCol= 'total_bill')

## Fit

In [0]:
linear_regressor = linear_regressor.fit(train_df)

In [0]:
linear_regressor.coefficients

Out[40]: DenseVector([2.8725, 3.8725, -1.2098, 2.0645, 0.1997, -1.845])

In [0]:
linear_regressor.intercept

Out[42]: 1.2773062564236712

## Predict

In [0]:
predicted_results = linear_regressor.evaluate(test_df)

In [0]:
predicted_results.predictions.show()

+--------------------+----------+------------------+
|independent features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07|12.612807215946766|
|(6,[0,1],[1.47,2.0])|     10.77|13.244746277550572|
|(6,[0,1],[1.97,2.0])|     12.02|14.680971417559217|
| (6,[0,1],[2.0,2.0])|     12.69|14.767144925959734|
| (6,[0,1],[2.0,3.0])|     16.31|18.639613980710475|
|(6,[0,1],[2.34,4.0])|     17.81|23.488716130667097|
|(6,[0,1],[2.72,2.0])|     13.28|16.835309127572184|
| (6,[0,1],[3.0,2.0])|      14.0|17.639595205977024|
|(6,[0,1],[7.58,4.0])|     39.42|  38.5403555979577|
| (6,[0,1],[9.0,4.0])|     48.33| 42.61923499558225|
|[1.1,2.0,1.0,1.0,...|      12.9|13.036583856630914|
|[1.17,2.0,0.0,1.0...|     32.83|14.447497179699974|
|[1.44,2.0,0.0,0.0...|      7.56|11.713066642869917|
|[1.5,2.0,1.0,0.0,...|      8.35|10.675571856203105|
|[1.64,2.0,0.0,1.0...|     15.36|  15.7975488113081|
|[1.68,2.0,1.0,0.0...|     13.42|11.1926129066

## Model Evaluation

In [0]:
predicted_results.meanAbsoluteError, predicted_results.r2, predicted_results.meanSquaredError

Out[47]: (3.943904737502374, 0.6636306386139149, 27.933762615069742)